In [75]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv

# Load the Cora dataset
dataset = Planetoid(root='data/Cora', name='Cora')

# Prepare data
data = dataset[0]

# Define a 2-layer GCN
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return torch.log_softmax(x, dim=1)

# Initialize model, optimizer, and loss function
model = GCN(input_dim=dataset.num_node_features, hidden_dim=16, output_dim=dataset.num_classes)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Training loop
for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

print("Training complete!")


Epoch 0, Loss: 1.9507317543029785
Epoch 10, Loss: 0.666307270526886
Epoch 20, Loss: 0.127598837018013
Epoch 30, Loss: 0.02733098715543747
Epoch 40, Loss: 0.009782478213310242
Epoch 50, Loss: 0.005362648516893387
Epoch 60, Loss: 0.0037494285497814417
Epoch 70, Loss: 0.0029849072452634573
Epoch 80, Loss: 0.0025219605304300785
Epoch 90, Loss: 0.002197231398895383
Training complete!


## Explanation:
GCN aggregates features from a node’s neighbors using graph convolutions. This allows the network to learn representations based on both node features and graph structure.
The Cora dataset is used to classify nodes into one of 7 research topics.

## Questions (1 point each):

1. What would happen if we added more GCN layers (e.g., 3 layers instead of 2)? How would this affect over-smoothing?
2. What would happen if we used a larger hidden dimension (e.g., 64 instead of 16)? How would this impact the model's capacity?
3. What would happen if we replaced ReLU activation with a sigmoid function? Would the performance change?

4. What would happen if we trained on only 10% of the nodes and tested on the remaining 90%? How would the performance be affected?
5. What would happen if we used a different optimizer (e.g., RMSprop) instead of Adam? Would it affect the convergence speed?

Extra credit: 
1. What would happen if we used edge weights (non-binary) in the adjacency matrix? How would it affect message passing?
2. What would happen if we removed the log-softmax function in the output layer? Would the loss function still work correctly?

## No points, just for you to think about:
1. What would happen if we applied dropout to the node features during training? How would it affect the model’s generalization?
2. What would happen if we used mean-pooling instead of summing the messages in the GCN layers?
3. What would happen if we pre-trained the node features using a different algorithm, like Node2Vec, before feeding them into the GCN?


### 1) What would happen if we added more GCN layers (e.g., 3 layers instead of 2)? How would this affect over-smoothing?

It is too my understanding that there is an upper-bound of how many GCN layers you can add before the network is affected by over-smoothing. My initial hypothesis is that you will eventually reach a certain depth in GCN layers where the accuracy will start to decrease on the test-set, even though the model will become stronger on the train set (which makes sense). 

To test this, I plan on training a GCN model with >2 convulutional layers until I find that the model starts to perform worse on the test-set. I will also need to write code to test the model on the testing set and find the accuracy to do the analysis

In [41]:
# Writing a function to test model accuracy on unseen testing data
def test(model, data):
    model.eval()
    with torch.no_grad():
        preds = model(data)[data.test_mask].argmax(1)
        actuals = data.y[data.test_mask]
        return float(sum(preds == actuals) / len(preds))

k = 5 

# Define a k-layer GCN 
class GCN_k(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        hidden_dim2 = int((2/3 * hidden_dim) + 10) # Found this hidden dimension general formula online
        hidden_dim3 = int((2/3 * hidden_dim2) + 10)
        hidden_dim4 = int((2/3 * hidden_dim3) + 10)
        super(GCN_k, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim2)
        self.conv3 = GCNConv(hidden_dim2, hidden_dim3)
        self.conv4 = GCNConv(hidden_dim3, hidden_dim4)
        self.conv5 = GCNConv(hidden_dim4, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        x = self.conv3(x, edge_index)
        x = torch.relu(x)
        x = self.conv4(x, edge_index)
        x = torch.relu(x)
        x = self.conv5(x, edge_index)
        return torch.log_softmax(x, dim=1)

# Initialize model with 2 GCN layers (same as the intitial given model)
model = GCN(input_dim=dataset.num_node_features, hidden_dim=16, output_dim=dataset.num_classes)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Initialize model with k GNC layers (everything is the same as the initial model but with more GCN layers)
model_k = GCN_k(input_dim=dataset.num_node_features, hidden_dim=16, output_dim=dataset.num_classes)
optimizer_k = optim.Adam(model_k.parameters(), lr=0.01)
criterion_k = nn.CrossEntropyLoss()
        
# Training loop
for epoch in range(100):
    model.train()
    optimizer.zero_grad()

    model_k.train()
    optimizer_k.zero_grad()
    
    # Forward pass
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    out_k = model_k(data)
    loss_k = criterion_k(out_k[data.train_mask], data.y[data.train_mask])
    loss_k.backward()
    optimizer_k.step()

    if epoch % 10 == 0:
        print(f'Model with 2 GCN Layers: Epoch {epoch}, Loss: {loss.item()}, Accuracy: {test(model, data)}')
        print(f'Model with {k} GCN Layers: Epoch {epoch}, Loss: {loss.item()}, Accuracy: {test(model_k, data)}')
        print('\n')

print("Training complete!")

Model with 2 GCN Layers: Epoch 0, Loss: 2.9703166484832764, Accuracy: 0.17299999296665192
Model with 5 GCN Layers: Epoch 0, Loss: 2.9703166484832764, Accuracy: 0.4059999883174896


Model with 2 GCN Layers: Epoch 10, Loss: 1.0429617166519165, Accuracy: 0.6549999713897705
Model with 5 GCN Layers: Epoch 10, Loss: 1.0429617166519165, Accuracy: 0.6629999876022339


Model with 2 GCN Layers: Epoch 20, Loss: 0.2774021029472351, Accuracy: 0.7770000100135803
Model with 5 GCN Layers: Epoch 20, Loss: 0.2774021029472351, Accuracy: 0.7490000128746033


Model with 2 GCN Layers: Epoch 30, Loss: 0.07073476165533066, Accuracy: 0.7860000133514404
Model with 5 GCN Layers: Epoch 30, Loss: 0.07073476165533066, Accuracy: 0.7239999771118164


Model with 2 GCN Layers: Epoch 40, Loss: 0.02347044087946415, Accuracy: 0.7799999713897705
Model with 5 GCN Layers: Epoch 40, Loss: 0.02347044087946415, Accuracy: 0.718999981880188


Model with 2 GCN Layers: Epoch 50, Loss: 0.011639274656772614, Accuracy: 0.7839999794960

As expected, you can see that by Epoch 90, the GCN with 5 layers has performed worse than the GCN with just 2 layers on the testing data. This is due to the oversmoothing of the network due to depth of training the model to conform to the training data. Although, this is probably not enough testing to say so for certain, the implication is there that it is in fact possible. 

After some additional research, I found a paper where it was proved that is indeed not possible to overcome the oversmoothing of GNNs with additional layers due to the fact that if you keep adding layers then eventually every node will be represented by the same feature vector which will lead to a very generalized and overfit model on the training data. 

This is the paper: https://papers.nips.cc/paper_files/paper/2023/file/6e4cdfdd909ea4e34bfc85a12774cba0-Paper-Conference.pdf

### 2) What would happen if we used a larger hidden dimension (e.g., 64 instead of 16)? How would this impact the model's capacity?

My initial thinking leads me to hypothesize that if you have a larger hidden dimension, then the model wil be likeley be overtrained on the training data and will in turn not be a good model to be used on unseen testing data. I think this because with a larger hidden dimension, the network is able to do more computations and recognize more patterns within the training data. If this dimension is too large, then the network will do a poor job of identifying the same patterns in other data. 

To test this, I'm going to run 2 models both with 2 GCN layers, where one model will have a hidden layer of dimension 64 and the other model will have a hidden layer of dimension 16. With this, I will be able to see whether my hypothesis is right based off of the model accuracy on testing data. 

In [40]:
# Initialize model with 2 GCN layers (same as the intitial given model)
model = GCN(input_dim=dataset.num_node_features, hidden_dim=16, output_dim=dataset.num_classes)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Initialize model with 2 GCN layers but with a 64 dimension hidden layer
model_hidden64 = GCN_k(input_dim=dataset.num_node_features, hidden_dim=64, output_dim=dataset.num_classes)
optimizer_hidden64 = optim.Adam(model_hidden64.parameters(), lr=0.01)
criterion_hidden64 = nn.CrossEntropyLoss()
        
# Training loop
for epoch in range(100):
    model.train()
    optimizer.zero_grad()

    model_hidden64.train()
    optimizer_hidden64.zero_grad()
    
    # Forward pass
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    out_hidden64 = model_hidden64(data)
    loss_hidden64 = criterion_hidden64(out_hidden64[data.train_mask], data.y[data.train_mask])
    loss_hidden64.backward()
    optimizer_hidden64.step()

    if epoch % 10 == 0:
        print(f'Model with hidden dimension 16: Epoch {epoch}, Loss: {loss.item()}, Accuracy: {test(model, data)}')
        print(f'Model with hidden dimension 64: Epoch {epoch}, Loss: {loss.item()}, Accuracy: {test(model_hidden64, data)}')
        print('\n')

print("Training complete!")

Model with hidden dimension 16: Epoch 0, Loss: 2.9961729049682617, Accuracy: 0.29100000858306885
Model with hidden dimension 64: Epoch 0, Loss: 2.9961729049682617, Accuracy: 0.4830000102519989


Model with hidden dimension 16: Epoch 10, Loss: 1.4488096237182617, Accuracy: 0.6840000152587891
Model with hidden dimension 64: Epoch 10, Loss: 1.4488096237182617, Accuracy: 0.8050000071525574


Model with hidden dimension 16: Epoch 20, Loss: 0.7292775511741638, Accuracy: 0.7059999704360962
Model with hidden dimension 64: Epoch 20, Loss: 0.7292775511741638, Accuracy: 0.7940000295639038


Model with hidden dimension 16: Epoch 30, Loss: 0.5014190673828125, Accuracy: 0.7229999899864197
Model with hidden dimension 64: Epoch 30, Loss: 0.5014190673828125, Accuracy: 0.7799999713897705


Model with hidden dimension 16: Epoch 40, Loss: 0.45806989073753357, Accuracy: 0.7319999933242798
Model with hidden dimension 64: Epoch 40, Loss: 0.45806989073753357, Accuracy: 0.7839999794960022


Model with hidden d

Based off of the results from above, my hypothesis is **incorrect** and the model does in fact perform better on testing data when the hidden layer dimension is larger. Time and time again, the model with 64 dimensions in the hidden layer performed better than the model with 16 dimensions hidden layers. This is due to the fact that with a higher dimension in the hidden layer, the model has more computational ability to find patterns that will help it recognize unseen data well.

### 3. What would happen if we replaced ReLU activation with a sigmoid function? Would the performance change?

If we replaced the ReLU activation function in the GCN layers with a sigmoid function, I would assume that the performance would decrease. This is due to the fact that ReLU is a simple activation problem where the output is either 0 or a positive number. With Sigmoid, the values can be in any range between 0 and 1 and can cause problems when training the layers of a neural network. Also, the ReLU is a lot faster computation wise than sigmoid. 

To test this out, I am going to train 2 models, one trained with ReLU activation and the other with sigmoid. I'm going to then evaluate the respective models on testing accuracy to see if the above statements are in fact true. 

In [49]:
# Creating a GCN with sigmoid activation

class GCN_sigmoid(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN_sigmoid, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.sigmoid(x)
        x = self.conv2(x, edge_index)
        return torch.log_softmax(x, dim = 1)

# Initialize model with ReLU Activation
model = GCN(input_dim=dataset.num_node_features, hidden_dim=16, output_dim=dataset.num_classes)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Initialize model with Sigmoid Activation
model_sigmoid = GCN_sigmoid(input_dim=dataset.num_node_features, hidden_dim=64, output_dim=dataset.num_classes)
optimizer_sigmoid = optim.Adam(model_sigmoid.parameters(), lr=0.01)
criterion_sigmoid = nn.CrossEntropyLoss()
        
# Training loop
for epoch in range(100):
    model.train()
    optimizer.zero_grad()

    model_sigmoid.train()
    optimizer_sigmoid.zero_grad()
    
    # Forward pass
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    out_sigmoid = model_sigmoid(data)
    loss_sigmoid = criterion_sigmoid(out_sigmoid[data.train_mask], data.y[data.train_mask])
    loss_sigmoid.backward()
    optimizer_sigmoid.step()

    if epoch % 10 == 0:
        print(f'Model with ReLU Activation: Epoch {epoch}, Loss: {loss.item()}, Accuracy: {test(model, data)}')
        print(f'Model with Sigmoid Activation: Epoch {epoch}, Loss: {loss.item()}, Accuracy: {test(model_sigmoid, data)}')
        print('\n')

print("Training complete!")

Model with ReLU Activation: Epoch 0, Loss: 2.9995131492614746, Accuracy: 0.2070000022649765
Model with Sigmoid Activation: Epoch 0, Loss: 2.9995131492614746, Accuracy: 0.16899999976158142


Model with ReLU Activation: Epoch 10, Loss: 1.5442249774932861, Accuracy: 0.46299999952316284
Model with Sigmoid Activation: Epoch 10, Loss: 1.5442249774932861, Accuracy: 0.781000018119812


Model with ReLU Activation: Epoch 20, Loss: 0.4719027280807495, Accuracy: 0.8019999861717224
Model with Sigmoid Activation: Epoch 20, Loss: 0.4719027280807495, Accuracy: 0.7929999828338623


Model with ReLU Activation: Epoch 30, Loss: 0.10823110491037369, Accuracy: 0.7860000133514404
Model with Sigmoid Activation: Epoch 30, Loss: 0.10823110491037369, Accuracy: 0.7820000052452087


Model with ReLU Activation: Epoch 40, Loss: 0.030787577852606773, Accuracy: 0.7889999747276306
Model with Sigmoid Activation: Epoch 40, Loss: 0.030787577852606773, Accuracy: 0.777999997138977


Model with ReLU Activation: Epoch 50, Los

As expected, the Sigmoid activation function does in fact reduce the performance of the model compared to the ReLU function, although not the extent I expected. I ran this experiment several times and sometimes the sigmoid activation did slightly better than the ReLU, but I am assuming that to be due to probability. After doing some more research, I learned that the reason the sigmoid function may be doing fine in our case is because the dataset is small and not too complex. In more complex situations, the ReLU would work far better. I also learned that sigmoid activation is better used on the output layer or if your network outpbinary, since all values in sigmoid are between 0 and 1. 

### 4) What would happen if we trained on only 10% of the nodes and tested on the remaining 90%? How would the performance be affected?

Based off the previous homework, and some general knowledge, the performance should decrease significantly when trained on only 10% of the data.

To test this, I will train a model using only 10% of the training nodes and test the results on the remaining 90% of the nodes

In [205]:
# Splitting the data into 10 percent training and 90% testing

# Load the Cora dataset
dataset = Planetoid(root='data/Cora', name='Cora')

# Prepare data
data = dataset[0]

total_nodes = data.num_nodes
ten_nodes = int(total_nodes * 0.10)
rest_nodes = total_nodes - ten_nodes

train_mask, test_mask = torch.zeros(total_nodes, dtype=torch.bool), torch.ones(total_nodes, dtype=torch.bool)

train_idxs = torch.randperm(total_nodes)[:ten_nodes]

for idx in train_idxs: 
    train_mask[idx] = True
    test_mask[idx] = False

data.train_mask = train_mask
data.test_mask = test_mask

In [201]:
# Initialize model, optimizer, and loss function
model = GCN(input_dim=dataset.num_node_features, hidden_dim=16, output_dim=dataset.num_classes)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Training loop
for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

print("Training complete!")

Epoch 0, Loss: 1.958348274230957
Epoch 10, Loss: 0.7300834655761719
Epoch 20, Loss: 0.18305355310440063
Epoch 30, Loss: 0.055140767246484756
Epoch 40, Loss: 0.02411525882780552
Epoch 50, Loss: 0.01451635081321001
Epoch 60, Loss: 0.01032477617263794
Epoch 70, Loss: 0.007950633764266968
Epoch 80, Loss: 0.006403629668056965
Epoch 90, Loss: 0.005307072773575783
Training complete!


When the model is being trained only on 10% of the data, the model loss is around 0.005 while it is 0.002 otherwise. After looking more into this problem, I am slightly confused on why the model doesn't perform better. When I took a deeper look into the original model, I saw that is used only 140 training nodes (20 from each class) and 10% of all nodes is around 271 nodes. Hence, the network is actually being trained on more data than the original model and I would assume that the model would do better in turn. The only justification I can see for why the model with more data performs worse is due to the unequal proportions of training data that the model leading to biases being built in the model. With the smaller training set of 140 nodes, there was 20 nodes from each class and this was uniform, when using 10% of all nodes this wasn't necessarily the case and there could be more nodes in a certain class than another which could lead to error. 

### 5) What would happen if we used a different optimizer (e.g., RMSprop) instead of Adam? Would it affect the convergence speed?

I am not too familiar with the different optimizer, so I will first run an experiment testing how the different optimizers affect convergence speed and then see the differences they make. 

To test this, I will time how long it takes to run each model with the two different optimizers (RMSprop and Adam) and compare the results. 



In [180]:
import time

# Load the Cora dataset
dataset = Planetoid(root='data/Cora', name='Cora')

# Prepare data
data = dataset[0]

# Initialize model, optimizer, and loss function
model = GCN(input_dim=dataset.num_node_features, hidden_dim=16, output_dim=dataset.num_classes)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

adam_start = time.time()
# Training loop
for epoch in range(400):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')
adam_end = time.time()
print("Training complete!")

model_rmsprop = GCN(input_dim=dataset.num_node_features, hidden_dim=16, output_dim=dataset.num_classes)
optimzer_rmsprop = optim.RMSprop(model.parameters(), lr = 0.01)
criterion_rmsprop = nn.CrossEntropyLoss()

rmsp_start = time.time()
# Training loop
for epoch in range(400):
    model_rmsprop.train()
    optimzer_rmsprop.zero_grad()
    
    # Forward pass
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimzer_rmsprop.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')
rmsp_end = time.time()
print("Training complete!")

print(f"Adam Time: {(adam_end - adam_start)}, RMSP Time: {(rmsp_end - rmsp_start)}")

Epoch 0, Loss: 1.951730728149414
Epoch 10, Loss: 0.561775267124176
Epoch 20, Loss: 0.09437353163957596
Epoch 30, Loss: 0.020149527117609978
Epoch 40, Loss: 0.007373816799372435
Epoch 50, Loss: 0.004144215025007725
Epoch 60, Loss: 0.0029777614399790764
Epoch 70, Loss: 0.0024123431649059057
Epoch 80, Loss: 0.002070226240903139
Epoch 90, Loss: 0.0018263454549014568
Epoch 100, Loss: 0.0016348736826330423
Epoch 110, Loss: 0.0014763379003852606
Epoch 120, Loss: 0.001341241761110723
Epoch 130, Loss: 0.001224307343363762
Epoch 140, Loss: 0.0011221718741580844
Epoch 150, Loss: 0.001032344182021916
Epoch 160, Loss: 0.0009528992231935263
Epoch 170, Loss: 0.0008823669631965458
Epoch 180, Loss: 0.0008194185793399811
Epoch 190, Loss: 0.0007629921892657876
Epoch 200, Loss: 0.000712250592187047
Epoch 210, Loss: 0.0006664280663244426
Epoch 220, Loss: 0.0006249368889257312
Epoch 230, Loss: 0.0005872652400285006
Epoch 240, Loss: 0.0005529700429178774
Epoch 250, Loss: 0.0005216271383687854
Epoch 260, Loss

Based off of the results above, the RMSprop optimzer is faster and converges faster than Adam! More specifically, in 400 epochs, RMSprop was able to get the loss down to 5.9423041420814116e-06 in 10.91 seconds, while Adam was able to get the loss down to 0.00026 in 11.242 seconds. Hence, RMSprop is the better optimzer for this problem since it is faster efficient, and converges faster!